In [1]:
import time

from helper_loader import *

set_log_level("DEBUG")

In [2]:
from scipy.spatial import KDTree

from histalign.backend.registration.alignment import interpolate_sparse_3d_array

In [3]:
def replace_workers_default(function: Callable, value: int) -> None:
    defaults = list(function.__defaults__)
    defaults[-1] = value
    function.__defaults__ = tuple(defaults)

In [13]:
alignment_path = Path(
    "/home/ediun/git/histalign/projects/project_cortical_depth/93e6cae680"
)

array = build_alignment_volume(alignment_path, return_raw_array=True)

mask = load_structure_mask("Isocortex", Resolution.MICRONS_25)

[2024-11-13 10:29:04] - [   DEBUG ] - Found cached volume. Loading from file. (histalign.backend.registration.alignment:55)


In [14]:
times = {}

In [17]:
for i in [-1]:  # [1, 2, 3, 4, 8]:
    replace_workers_default(KDTree.query, i)

    start_time = time.perf_counter()

    interpolated_array = interpolate_sparse_3d_array(
        array,
        reference_mask=mask,
        neighbours=16,
        use_cache=False,
    )

    times[i] = time.perf_counter() - start_time

[2024-11-13 10:42:07] - [    INFO ] - Starting interpolation with parameters {kernel: multiquadric, neighbours: 16, epsilon: 1, degree: None, chunk size: 1,000,000, recursive: False}. (histalign.backend.registration.alignment:204)
[2024-11-13 10:42:08] - [    INFO ] - Interpolating chunk 1/8 (12%). (histalign.backend.registration.alignment:238)
[2024-11-13 10:42:13] - [    INFO ] - Interpolating chunk 2/8 (25%). (histalign.backend.registration.alignment:238)
[2024-11-13 10:42:21] - [    INFO ] - Interpolating chunk 3/8 (38%). (histalign.backend.registration.alignment:238)
[2024-11-13 10:42:28] - [    INFO ] - Interpolating chunk 4/8 (50%). (histalign.backend.registration.alignment:238)
[2024-11-13 10:42:35] - [    INFO ] - Interpolating chunk 5/8 (62%). (histalign.backend.registration.alignment:238)
[2024-11-13 10:42:41] - [    INFO ] - Interpolating chunk 6/8 (75%). (histalign.backend.registration.alignment:238)
[2024-11-13 10:42:46] - [    INFO ] - Interpolating chunk 7/8 (88%). (his

In [18]:
for i, time_ in times.items():
    print(f"{i} workers: {time_:.2f}")

1 workers: 268.92
2 workers: 165.00
3 workers: 121.75
4 workers: 100.50
8 workers: 77.66
-1 workers: 50.73


Somatomotor areas (2 chunks):
```text
1 workers: 14.78
2 workers: 11.35
3 workers: 10.25
4 workers: 9.69
8 workers: 8.40
```

Olfactory areas (3 chunks):
```text
1 workers: 30.51
2 workers: 21.20
3 workers: 19.59
4 workers: 16.10
8 workers: 14.45
```

Isocortex (8 chunks):
```text
1 workers:  268.92
2 workers:  165.00
3 workers:  121.75
4 workers:  100.50
8 workers:  77.66
-1 workers: 50.73
```